In [0]:
#authorization from google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#load data
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/train_clean.csv")


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df.head()

,Unnamed: 0,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,indresi,indext,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,1,A,S,N,KHL,N,1.0,29.0,MALAGA,1.0,87218.10,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1,1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,1,I,S,S,KHE,N,1.0,13.0,CIUDAD REAL,0.0,35548.74,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,1,I,S,N,KHE,N,1.0,13.0,CIUDAD REAL,0.0,122179.11,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,1,I,S,N,KHD,N,1.0,50.0,ZARAGOZA,0.0,119775.54,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
4,4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,1,A,S,N,KHE,N,1.0,50.0,ZARAGOZA,1.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


In [0]:
df['fecha_dato'].unique()

array(['2015-01-28', '2015-02-28', '2015-03-28', '2015-04-28',
       '2015-05-28', '2015-06-28', '2015-07-28', '2015-08-28',
       '2015-09-28', '2015-10-28', '2015-11-28', '2015-12-28',
       '2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28',
       '2016-05-28'], dtype=object)

In [0]:
#delete first column
del df['Unnamed: 0']
#convert fecha_dato to datetime
#df['fecha_dato'] = pd.to_datetime(df['fecha_dato'], format= '%Y-%M-%d')

#Similarity Model


In [0]:
df.rename(columns={"Customer code":'ID', 'ncodpers':"Customer code", 'ind_empleado':"Employee index",
       'pais_residencia':"Customer's Country residence", 'sexo':"Customer's sex", 'age':"Age", 'ind_nuevo':"New customer Index",
       'antiguedad':"Customer seniority", 'indrel':"First/Primary", 'indrel_1mes':"Customer type at the beginning of the month", 'tiprel_1mes':"Customer relation type at the beginning of the month", 'indresi':"Residence index",
       'indext':"Foreigner index", 'canal_entrada':"channel", 'indfall':"Deceased index", 'tipodom':"Addres type", 'cod_prov':"Province code", 'nomprov':"Province name",
       'ind_actividad_cliente':"Activity index", 'renta':"Gross income of the household", 'segmento':"segmentation", 'ind_ahor_fin_ult1':"Saving Account",
       'ind_aval_fin_ult1':"Guarantees", 'ind_cco_fin_ult1':"Current Accounts", 'ind_cder_fin_ult1':"Derivada Account",
       'ind_cno_fin_ult1':"Payroll Account", 'ind_ctju_fin_ult1':"Junior Account", 'ind_ctma_fin_ult1':"Más particular Account",
       'ind_ctop_fin_ult1':"particular Account", 'ind_ctpp_fin_ult1':"particular Plus Account", 'ind_deco_fin_ult1':"Short-term deposits",
       'ind_deme_fin_ult1':"Medium-term deposits", 'ind_dela_fin_ult1':"Long-term deposits", 'ind_ecue_fin_ult1':"e-account",
       'ind_fond_fin_ult1':"Funds", 'ind_hip_fin_ult1':"Mortgage", 'ind_plan_fin_ult1':"Pensions",
       'ind_pres_fin_ult1':"Loans", 'ind_reca_fin_ult1':"Taxes", 'ind_tjcr_fin_ult1':"Credit Card",
       'ind_valo_fin_ult1':"Securities", 'ind_viv_fin_ult1':"Home Account", 'ind_nomina_ult1':"Payroll",
       'ind_nom_pens_ult1':"Pensions", 'ind_recibo_ult1':"Direct Debit"}, 
                 inplace=True)

In [0]:
df.head()

,fecha_dato,Customer code,Employee index,Customer's Country residence,Customer's sex,Age,fecha_alta,New customer Index,Customer seniority,First/Primary,Customer type at the beginning of the month,Customer relation type at the beginning of the month,Residence index,Foreigner index,channel,Deceased index,Addres type,Province code,Province name,Activity index,Gross income of the household,segmentation,Saving Account,Guarantees,Current Accounts,Derivada Account,Payroll Account,Junior Account,Más particular Account,particular Account,particular Plus Account,Short-term deposits,Medium-term deposits,Long-term deposits,e-account,Funds,Mortgage,Pensions,Loans,Taxes,Credit Card,Securities,Home Account,Payroll,Pensions,Direct Debit
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,1,A,S,N,KHL,N,1.0,29.0,MALAGA,1.0,87218.10,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,1,I,S,S,KHE,N,1.0,13.0,CIUDAD REAL,0.0,35548.74,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,1,I,S,N,KHE,N,1.0,13.0,CIUDAD REAL,0.0,122179.11,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,1,I,S,N,KHD,N,1.0,50.0,ZARAGOZA,0.0,119775.54,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,1,A,S,N,KHE,N,1.0,50.0,ZARAGOZA,1.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


In [0]:
! pip install turicreate

In [0]:
! pip install numpy cython
! git clone https://github.com/NicolasHug/surprise.git
! cd surprise
! python setup.py install

Cloning into 'surprise'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 4234 (delta 2), reused 2 (delta 0), pack-reused 4221
Receiving objects: 100% (4234/4234), 6.34 MiB | 22.69 MiB/s, done.
Resolving deltas: 100% (2956/2956), done.
python3: can't open file 'setup.py': [Errno 2] No such file or directory


In [0]:
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split
import surprise

In [0]:
customers = df[['fecha_dato','Customer code']]
customers.head()

,fecha_dato,Customer code
0,2015-01-28,1375586
1,2015-01-28,1050611
2,2015-01-28,1050612
3,2015-01-28,1050613
4,2015-01-28,1050614


In [0]:
train = df.loc[:, 'Saving Account':]
train = pd.concat([customers, train], axis=1)
train['Payroll'] = train['Payroll'].astype(np.int64) 
train['Pensions'] = train['Pensions'].astype(np.int64) 


In [0]:
train.head()

,fecha_dato,Customer code,Saving Account,Guarantees,Current Accounts,Derivada Account,Payroll Account,Junior Account,Más particular Account,particular Account,particular Plus Account,Short-term deposits,Medium-term deposits,Long-term deposits,e-account,Funds,Mortgage,Pensions,Loans,Taxes,Credit Card,Securities,Home Account,Payroll,Pensions,Direct Debit
0,2015-01-28,1375586,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2015-01-28,1050611,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2015-01-28,1050612,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2015-01-28,1050613,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2015-01-28,1050614,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Create the first month training data 

In [0]:
def training_data(train,month):
    train_month = train[train['fecha_dato'] == month]
    train_month['Payroll'] = train_month['Payroll'].astype(np.int64) 
    train_month['Pensions'] = train_month['Pensions'].astype(np.int64) 
    cols = list(train_month.columns)[2:]
    cols_all = list(train_month.columns)
    train_month = train_month[cols_all].replace({ 0.0:np.nan})
    train_month['Customer code'] = train_month['Customer code'].astype(np.int64)
    data = pd.melt(train_month.set_index('Customer code')[cols].apply(pd.Series).reset_index(), 
             id_vars=['Customer code'],
             value_name='products')\
    .groupby(['Customer code', 'variable']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'})
    data = data[data['purchase_count'] > 0]
    df_matrix = pd.pivot_table(data, values='purchase_count', index='Customer code', columns='variable')
    d = df_matrix.reset_index() 
    d.index.names = ['scaled_purchase_freq'] 
    data_norm = pd.melt(d, id_vars=['Customer code'], value_name='scaled_purchase_freq').dropna()  
    return data_norm     



In [0]:
train_data_2015_01 = training_data(train, '2015-01-28')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [0]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [0]:
user_id = 'Customer code'
item_id = 'variable'
name = 'pearson'
target = 'scaled_purchase_freq'
users_to_recommend = list(train_data_2015_01['Customer code'])
n_rec = 5 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset
test = tc.SFrame(train_data_2015_01)
pearson = model(test, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)
pop = model(test, 'popularity', user_id, item_id, target, users_to_recommend, n_rec, n_display)
cos = model(test, 'cosine', user_id, item_id, target, users_to_recommend, n_rec, n_display)


Preparing data set.

Data has 1101562 observations with 603591 users and 23 items.

Data prepared in: 1.00855s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 1098000/1101562 queries. users per second: 389383

recommendations finished on 1099000/1101562 queries. users per second: 389386

recommendations finished on 1100000/1101562 queries. users per second: 389399

recommendations finished on 1101000/1101562 queries. users per second: 389407

+---------------+-------------------------+---------------------+------+
| Customer code |         variable        |        score        | rank |
+---------------+-------------------------+---------------------+------+
|     15889     |       Direct Debit      |  0.2892865985631943 |  1   |
|     15889     |     Payroll Account     | 0.22790992259979248 |  2   |
|     15889     |         Pensions        | 0.22543688118457794 |  3   |
|     15889     |         Payroll         |  0.2030072808265686 |  4   |
|     15889     |        e-account        | 0.20291467010974884 |  5   |
|     15890     |          Taxes          |  0.6459851520402091 |  1   |
|     15890     |        Securities       |  0.3543805054255894 |  2   |
|     15890     |    Long-term deposits   | 0.35008877515792847 |  3   |
|     15890     |         Mortgage        |  0.3337094954081944 |  4   |
|     15890     |          Funds          |  0.283635607787541  |  5   |
|     15892     |         Pensions        |  0.3928

In [0]:
train_data_2015_02 = training_data(train, '2015-02-28')
train_data_2015_02 = tc.SFrame(train_data_2015_02)
models_w_norm = [pop,cos,pearson]
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [0]:
eval_norm = tc.recommender.util.compare_models(train_data_2015_02, models_w_norm, model_names=names_w_norm)


PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/606407 queries. users per second: 35125.9

recommendations finished on 2000/606407 queries. users per second: 59616.1

recommendations finished on 3000/606407 queries. users per second: 77170.4

recommendations finished on 4000/606407 queries. users per second: 91508.1

...

recommendations finished on 603000/606407 queries. users per second: 171565

recommendations finished on 604000/606407 queries. users per second: 171512

recommendations finished on 605000/606407 queries. users per second: 171358

recommendations finished on 606000/606407 queries. users per second: 171288


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |  0.009848171277706177 |  0.002117576008955949 |
|   2    |  0.004984276236916746 |  0.002132479217217689 |
|   3    | 0.0035779050483694588 | 0.0022591516985133904 |
|   4    |  0.003709967068322061 | 0.0031558786385165105 |
|   5    |  0.004169476935457768 |  0.004781951399487234 |
|   6    |  0.003789808934703394 |  0.00529039115788618  |
|   7    | 0.0032936389492297653 | 0.0054304465259072295 |
|   8    | 0.0029742812995232583 |  0.005887495240190819 |
|   9    | 0.0026573645166435254 |  0.005912000936341135 |
|   10   | 0.0025908342087080484 |  0.006310695164303474 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.201189956167278

Per User RMSE (best)
+---------------+------+-------+
| Customer c